# 문서 청킹 전략 실험

문서를 적절한 크기의 청크로 분할하는 전략을 테스트합니다.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# 프로젝트 루트 경로
project_root = Path().absolute().parent
sys.path.append(str(project_root))

from src import config
from src.chunking import DocumentChunker, chunk_text

# 한글 폰트 설정 (Windows)
plt.rc font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

print(f"기본 청킹 설정:")
print(f"  Chunk Size: {config.CHUNK_SIZE}")
print(f"  Chunk Overlap: {config.CHUNK_OVERLAP}")

## 1. 이전 노트북에서 로드한 문서 불러오기

In [ ]:
# 이전 노트북에서 저장한 문서 불러오기
try:
    %store -r loaded_docs
    print(f"✅ {len(loaded_docs)}개 문서 로드 완료")
except:
    print("❌ 이전 노트북에서 저장한 문서가 없습니다.")
    print("   02_document_loading.ipynb를 먼저 실행해주세요.")
    loaded_docs = []

## 2. 기본 청킹 테스트

In [ ]:
if loaded_docs:
    # 첫 번째 문서로 테스트
    test_doc = loaded_docs[0]
    
    print(f"테스트 문서: {test_doc['file_name']}")
    print(f"원본 텍스트 길이: {test_doc['char_count']:,} 글자")
    
    # 청킹
    chunker = DocumentChunker()
    chunks = chunker.chunk_document(test_doc)
    
    print(f"\n생성된 청크 수: {len(chunks)}개")
    print(f"\n청크 크기 분포:")
    chunk_sizes = [chunk['chunk_size'] for chunk in chunks]
    print(f"  평균: {np.mean(chunk_sizes):.0f} 글자")
    print(f"  중앙값: {np.median(chunk_sizes):.0f} 글자")
    print(f"  최소: {np.min(chunk_sizes):.0f} 글자")
    print(f"  최대: {np.max(chunk_sizes):.0f} 글자")

In [ ]:
# 청크 샘플 출력
print("\n청크 샘플 (처음 3개):\n")
for i, chunk in enumerate(chunks[:3]):
    print(f"[청크 {i}] ({chunk['chunk_size']} 글자)")
    print("-" * 60)
    print(chunk['text'][:300])
    print("...\n")

## 3. 다양한 청킹 설정 비교

In [ ]:
if loaded_docs:
    test_doc = loaded_docs[0]
    
    # 다양한 chunk_size 테스트
    chunk_sizes_to_test = [500, 1000, 1500, 2000]
    overlap = 200
    
    results = []
    
    for chunk_size in chunk_sizes_to_test:
        chunker = DocumentChunker(chunk_size=chunk_size, chunk_overlap=overlap)
        chunks = chunker.chunk_document(test_doc)
        
        results.append({
            'chunk_size': chunk_size,
            'num_chunks': len(chunks),
            'avg_size': np.mean([c['chunk_size'] for c in chunks])
        })
        
        print(f"Chunk Size {chunk_size} → {len(chunks)}개 청크 (평균 {results[-1]['avg_size']:.0f} 글자)")

In [ ]:
# 시각화
import pandas as pd

if results:
    df_results = pd.DataFrame(results)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 청크 개수
    ax1.bar(df_results['chunk_size'].astype(str), df_results['num_chunks'])
    ax1.set_xlabel('Chunk Size')
    ax1.set_ylabel('청크 개수')
    ax1.set_title('Chunk Size별 청크 개수')
    
    # 평균 크기
    ax2.bar(df_results['chunk_size'].astype(str), df_results['avg_size'])
    ax2.set_xlabel('Chunk Size')
    ax2.set_ylabel('평균 크기 (글자)')
    ax2.set_title('Chunk Size별 실제 평균 크기')
    
    plt.tight_layout()
    plt.show()

## 4. 전체 문서 청킹

In [ ]:
if loaded_docs:
    print(f"{len(loaded_docs)}개 문서 청킹 중...\n")
    
    chunker = DocumentChunker()  # 기본 설정 사용
    all_chunks = chunker.chunk_documents(loaded_docs)
    
    print(f"\n✅ 총 {len(all_chunks)}개 청크 생성 완료")
    
    # 청크 통계
    chunk_sizes = [chunk['chunk_size'] for chunk in all_chunks]
    print(f"\n청크 크기 통계:")
    print(f"  평균: {np.mean(chunk_sizes):.0f} 글자")
    print(f"  중앙값: {np.median(chunk_sizes):.0f} 글자")
    print(f"  표준편차: {np.std(chunk_sizes):.0f}")
    print(f"  최소: {np.min(chunk_sizes):.0f} 글자")
    print(f"  최대: {np.max(chunk_sizes):.0f} 글자")

In [ ]:
# 청크 크기 분포 시각화
if all_chunks:
    plt.figure(figsize=(10, 6))
    plt.hist(chunk_sizes, bins=50, edgecolor='black')
    plt.xlabel('청크 크기 (글자)')
    plt.ylabel('빈도')
    plt.title('청크 크기 분포')
    plt.axvline(config.CHUNK_SIZE, color='r', linestyle='--', label=f'목표 크기: {config.CHUNK_SIZE}')
    plt.legend()
    plt.show()

## 5. 청크 품질 검증

In [ ]:
# 너무 작거나 큰 청크 확인
min_threshold = 100  # 최소 100글자
max_threshold = config.CHUNK_SIZE * 1.5  # 목표 크기의 1.5배

too_small = [c for c in all_chunks if c['chunk_size'] < min_threshold]
too_large = [c for c in all_chunks if c['chunk_size'] > max_threshold]

print(f"품질 검증:")
print(f"  너무 작은 청크 ({min_threshold}자 미만): {len(too_small)}개 ({len(too_small)/len(all_chunks)*100:.1f}%)")
print(f"  너무 큰 청크 ({max_threshold:.0f}자 초과): {len(too_large)}개 ({len(too_large)/len(all_chunks)*100:.1f}%)")
print(f"  적정 크기 청크: {len(all_chunks)-len(too_small)-len(too_large)}개 ({(len(all_chunks)-len(too_small)-len(too_large))/len(all_chunks)*100:.1f}%)")

In [ ]:
# 너무 작은 청크 샘플 확인
if too_small:
    print("\n너무 작은 청크 샘플:")
    for chunk in too_small[:3]:
        print(f"\n파일: {chunk.get('file_name', 'Unknown')}")
        print(f"크기: {chunk['chunk_size']} 글자")
        print(f"내용: {chunk['text'][:100]}...")

## 6. 청킹 결과 저장

In [ ]:
# 다음 노트북에서 사용할 수 있도록 저장
%store all_chunks

print("✅ 청킹 결과 저장 완료")
print(f"   {len(all_chunks)}개 청크")
print("\n다음 단계: 04_build_vectordb.ipynb")

## 7. 청킹 전략 결론

### 베이스라인 설정
- **Chunk Size**: 1000
- **Chunk Overlap**: 200

### 향후 개선 방향
1. **의미 기반 청킹**: 문서 구조(제목, 섹션 등)를 고려한 청킹
2. **동적 크기 조정**: 문서 특성에 따라 청크 크기 자동 조정
3. **Overlap 최적화**: 검색 성능에 따라 overlap 크기 조정